# 第3章 ニューラルネットワーク

## サマリー

* パーセプトロンでは入力信号の線形関数に活性化関数で変換して出力結果を出す。
* 単純パーセプトロンの活性化関数は専らステップ関数
* 多層パーセプトロンの活性化関数はシグモイド関数かReLU関数


## ニューラルネットワークを実装しよう

### 第1層（活性化関数なし）
![第1層（活性化関数なし）](multilayer_perceptron_l0tol1_noActFnc.png)

In [ ]:
import numpy as np

#==========第1層==========
# 入力データ
X = np.array([0.5, 0.2])

# 重み
W1 = np.array([[0.2, 0.1, 0.5],[0.4, 0.2, 0.6]])

# バイアス
B1 = np.array([0.2, 0.4, 0])


# 出力結果
A1 = np.dot(X, W1) + B1
print(f"A1 : {A1}")

### 第1層
![第1層](multilayer_perceptron_l0tol1.png)

In [ ]:
#====活性化関数：シグモイド関数==== 
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 出力結果
Z1 = sigmoid(A1)
print(f"Z1 : {Z1}")

### 第2層
![第2層](multilayer_perceptron_l1tol2.png)

In [ ]:
#==========第2層===========
# 重み
W2 = np.array([[0.3, 0.2], [0.2, 0.7], [1.0, 0.6]])

# バイアス
B2 = np.array([1.0, 0.2])


# 出力結果
A2 = np.dot(Z1, W2) + B2
print(f"A2 : {A2}")

Z2 = sigmoid(A2)
print(f"Z2 : {Z2}")

### 出力層
![出力層](multilayer_perceptron_l2tol3.png)

In [ ]:
#==========出力層==========
# 重み
W3 = np.array([[0.5, 0], [0.1, 0.7]])

# バイアス
B3 = np.array([1.0, 0.2])


# 出力結果
A3 = np.dot(Z2, W3) + B3
print(f"A3 : {A3}")

y = sigmoid(A3)
print(f"y : {y}")

## 出力層の設計

* 分類問題⇒ソフトマックス関数
* 回帰問題⇒恒等関数

### ソフトマックス関数


■定義<br>
![ソフトマックス関数](soft_max_fnc.png)<br>
* 滑らかな（＝ソフトな）曲線となり、1つの出力値が最大となるため、「ソフトマックス関数」と呼ばれるらしい。
* n=2のときシグモイド関数になる
* C^∞級関数
* 0~1なので出力結果を確率っぽく扱える（Aが70%、Bが25%みたいな説明が出来る）（※本当に確率の定義を満たしているのかは要確認）
* 0~1かつ単調増加なので、__元の出力値の大小関係を保存しつつ正規化が可能__（元の出力値が負になる場合も考慮すると単純に出力値を足し合わせたり、絶対値を足したりすることが出来ないことが分かる）
* 指数関数の計算はコンピュータにとってそれなりに負担なため、学習時には使用せず予測時に使用する

#### ソフトマックス関数におけるオーバーフローの考慮

* 指数関数を使用するので容易にオーバーフローが起こる
* 大きな数同士での掛け算、割り算は計算が不安定になるので極力避ける
* 指数部を同じ数で足し引きしてもソフトマックス関数の出力値は変わらないため、オーバーフロー防止として入力の最大値で入力値を引く

In [ ]:
import numpy as np

# ソフトマックス関数
def softmax(X):
    exp_X = np.exp(X)
    sum_exp_X = np.sum(exp_X)
    return exp_X / sum_exp_X

y = softmax([1000, 1001, 1002])
print(y)
# > [nan nan nan]


# ソフトマックス関数（オーバーフロー抑止）
def softmax(X):
    c = np.max(X)
    exp_X = np.exp(X-c)
    sum_exp_X = np.sum(exp_X)
    return exp_X / sum_exp_X

y = softmax([1000, 1001, 1002])
print(y)
# > [0.09003057 0.24472847 0.66524096]

### 高速化とバッチ処理

バッチ処理により同時に複数処理を行う方が高速になる。理由は以下：
* nupmyには大きな配列の計算最適化の仕組みがある
* データ読込のオーバーヘッドが短縮できる